In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import torch
from torch import nn

from hparams.hparams_GT import create_hparams
from datasets import LJDataset, TextMelCollate

In [2]:
!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Sat Mar 13 17:10:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:19:00.0 Off |                  N/A |
| 75%   65C    P2   261W / 370W |  11453MiB / 24268MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:68:00.0 Off |                  N/A |
| 67%   

In [3]:
from parallel_wavegan.utils import download_pretrained_model
from parallel_wavegan.utils import load_model

device='cuda'
vocoder_tag = "ljspeech_parallel_wavegan.v3"

# NOTE: Sometimes download is failed due to "Permission denied". That is 
#   the limitation of google drive. Please retry after serveral hours.
vocoder = load_model(download_pretrained_model(vocoder_tag)).to(device).eval()
vocoder.remove_weight_norm()
print('done')

from parallel_wavegan.utils import read_hdf5

stats_filename = "/home/scpark/.cache/parallel_wavegan/ljspeech_parallel_wavegan.v3/stats.h5"
mean = read_hdf5(stats_filename, 'mean')
scale = read_hdf5(stats_filename, 'scale')
print('done')

done
done


In [4]:
stt_hparams, tts_hparams = create_hparams()
testset = LJDataset(tts_hparams, split='test')
collate_fn = TextMelCollate(tts_hparams)

test_loader = torch.utils.data.DataLoader(testset, num_workers=1, 
                          shuffle=False, sampler=None, batch_size=1, pin_memory=False,
                          drop_last=True, collate_fn=collate_fn)
print(test_loader)

In [5]:
def to_cuda(batch):
    batch['text'] = batch['text'].cuda()
    batch['text_lengths'] = batch['text_lengths'].cuda()
    batch['mels'] = batch['mels'].cuda()
    batch['mel_lengths'] = batch['mel_lengths'].cuda()
    
    return batch

In [6]:
!mkdir inference

mkdir: cannot create directory ‘inference’: File exists


In [ ]:
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import time 
import soundfile as sf

inference_dir = 'inference/GT/'
!mkdir $inference_dir

vocoder.eval()
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        print(i, batch['strings'][0])
        batch = to_cuda(batch)
        _s = batch['mels'].data.cpu().numpy()
        _mel = _s[0].T
        _mel = (_mel - mean) / scale
        mel = torch.from_numpy(_mel).cuda()
        wav = vocoder.inference(mel)
        _wav = wav[:, 0].data.cpu().numpy()
        save_path = inference_dir + 'inference_' + str(i) + '.wav'
        sf.write(save_path, _wav, 22050, subtype='PCM_16')
        
print('done')

mkdir: cannot create directory ‘inference/GT/’: File exists
0 The Chronicles of Newgate, Volume two. By Arthur Griffiths. Section five: Newgate down to eighteen eighteen, part two.
1 The squalor and uncleanness of the debtors' side was intensified by constant overcrowding.
2 Prisoners were committed to it quite without reference to its capacity.
3 No remonstrance was attended to,
4 no steps taken to reduce the number of committals, and the governor was obliged to utilize the chapel as a day and night room.
5 Besides this, although the families of debtors were no longer permitted to live with them inside the jail,
6 hundreds of women and children came in every morning to spend the day there, and there was no limitation whatever to the numbers of visitors admitted to the debtors' side.
7 Friends arrived about nine a.m., and went out at nine p.m., when as many as two hundred visitors have been observed leaving the debtors' yards at one time.
8 The day passed in revelry and drunkenness. Al

78 Mr. Bennet refers to a gentleman confined for want of bail, who occupied a room with five others
79 two committed by the Bankruptcy Commissioner, one for perjury, and two transports.
80 Persons convicted of publishing libels were still immured in the same rooms with transports and felons.
81 The middle yard, as far as its limits would permit, was appropriated to felons and transports. The wards here were generally very crowded.
82 Each ward was calculated to hold twenty-four, allowing each individual one foot and a half;
83 Quote, a common-sized man, says the keeper, Mr. Newman, can turn in nineteen inches, end quote.
84 These twenty-four could just sleep on the barrack bed; when the number was higher, and it often rose to forty, the surplus had to sleep on the floor.
85 The crowding was in consequence of the delay in removing transports.
86 These often remained in Newgate for six months, sometimes a year, in some cases longer;
87 in one, for seven years -- that of a man sentenced t

158 To constitute this the aristocratic quarter, unwarrantable demands were made upon the space properly allotted to the female felons,
159 and no lodger was rejected, whatever his status, who offered himself and could bring grist to the mill.
160 The luxury of the state side was for a long time open to all who could pay
161 the convicted felon, the transport awaiting removal, the lunatic whose case was still undecided,
162 the misdemeanant tried or untried, the debtor who wished to avoid the discomfort of the crowded debtors' side, the outspoken newspaper editor,
163 or the daring reporter of parliamentary debates.
164 The better class of inmate complained bitterly of this enforced companionship with the vile,
165 association at one time forbidden by custom, but which greed and rapacity long made the rule.
166 The fee for admission to the state side, as fixed by the table of fees, was three guineas, but Mr. Newman declared that he never took more than two.
167 Ten and sixpence a week 